<a href="https://colab.research.google.com/github/mirjunaid26/datascientist/blob/main/ConvertingTabularDatasetToGraphDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Converting Tabular Dataset To Graph Dataset

I hope this notebook helps you to convert your CSV file into a graph dataset 🚀


`Step 0`

Bring some creativity and don't lose hope- It is very natural that it takes some time to rearrange the data in a graph format. Also, this notebook is only to help you to get started (you will have to transfer it to your specific use-case).

`Step 1`

To get started, identify the following things in your dataset (I have some real-world examples below in this notebook):

- Nodes (Items, People, Locations, Cars, ...)
- Edges (Connections, Interactions, Similarity, ...)
- Node Features (Attributes)
- Labels (Node-level, edge-level, graph-level)

and optionally:
- Edge weights (Strength of the connection, number of interactions, ...)
- Edge features (Additional (multi-dim) properties describing the edge)



`Step 2`

Do you have different node and edge types? (This means the nodes/edges have different attributes such as Cars vs. People)

- No, all my edges/nodes have the same type  --> **Proceed with 1.1**
- Yes, there are different relations and node types --> **Proceed with 1.2**


## 1.1 Homogeneous